In [2]:
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.stats import multivariate_normal, invwishart
from permutation_helpers import random_data_gen
# def random_data_gen(n_samples=1000, n_feats=10, maha=1.0, psi_diag=1.0, psi_offdiag=0., ddof=150, class_ratio=0.5, seed=None):
#     if seed:
#         np.random.seed(seed)
#     ## initialize multivariate normal dist with normally distributed means and covariance
#     ## drawn from an inverse wishart distribution (conjugate prior for MVN)
#     norm_means_a = np.random.randn(n_feats)
#     norm_means_b = np.zeros_like(norm_means_a)
#     psi = psi_diag * np.eye(n_feats) + psi_offdiag * ~np.eye(n_feats).astype(bool)
#     nu = n_feats + ddof
#     wishart_cov = invwishart(nu, psi).rvs()
#     ## specify the mahalanobis distance between the two distributions
#     dist = mahalanobis(norm_means_a, norm_means_b, np.linalg.inv(wishart_cov))
#     norm_means_a = norm_means_a * (maha / dist)
#     assert np.isclose(mahalanobis(norm_means_a, norm_means_b, np.linalg.inv(wishart_cov)), maha)
#     ## multivariate normal distributions with different means and equal variances
#     mvn_a = multivariate_normal(mean=norm_means_a, cov=wishart_cov)
#     mvn_b = multivariate_normal(mean=norm_means_b, cov=wishart_cov)
#     ## not used, but compute correlations
#     corr = (D:=np.diag(1/np.sqrt(np.diag(wishart_cov)))) @ wishart_cov @ D
#     ## generate data samples from a multivariate normal
#     data = np.vstack([mvn_a.rvs(int(n_samples*class_ratio)), mvn_b.rvs(n_samples - int(n_samples*class_ratio))])
#     labels = np.arange(len(data))<int(n_samples*class_ratio)
#     return data, labels
# #     idx = np.random.choice(np.arange(n_samples), n_samples, replace=False)
# #     return data[idx], labels[idx]

In [3]:
data, labels = random_data_gen(n_samples=1000, n_feats=10, maha=1., psi_diag=1., seed=1)
data, labels

(array([[-0.05388803, -0.04537587, -0.12044986, ..., -0.1208464 ,
          0.0104293 ,  0.01338025],
        [ 0.04694033,  0.01004464, -0.01491472, ..., -0.05995696,
          0.02228193, -0.0760042 ],
        [-0.07455321,  0.05215185,  0.00418394, ..., -0.0290755 ,
         -0.03820365,  0.00456696],
        ...,
        [ 0.01424548,  0.09464702, -0.11352255, ...,  0.08822633,
          0.08589731, -0.06343076],
        [-0.00605004,  0.0354669 ,  0.09935671, ...,  0.03441683,
          0.08805234,  0.08118872],
        [ 0.03475752,  0.13853059, -0.08067336, ..., -0.1465611 ,
         -0.06162906, -0.0460488 ]]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  Tr

In [6]:
np.save("random_data_X.npy", data)
np.save("random_data_y", labels)

In [4]:
def E_invwish(psi, dof):
    n_feats = len(psi)
    return psi / (dof-n_feats-1)

def Var_invwish(psi, dof):
    p = len(psi)
    Var = np.empty((p, p))
    for i in range(p):
        for j in range(p):
            Var[i][j] = (dof-p+1) * psi[i][j]**2 + (dof-p-1) * psi[i][i]*psi[j][j] 
    Var /= (dof-p)*(dof-p-1)**2*(dof-p-3)
    return Var

In [5]:
def covariance_invwishart(diag=1., offdiag=0., ddof=4, p=5):
    psi = diag*np.eye(5) + offdiag * np.ones((p, p))
    expected = E_invwish(psi, p+ddof)
    variance = Var_invwish(psi, p+ddof)
    return expected, variance 

expected, variance = covariance_invwishart(1, 0, ddof=11)
print(f"Expected Cov:\n{expected}")
corr = (D:=np.diag(1/np.sqrt(np.diag(expected)))) @ expected @ D
print(f"Expected Corr:\n{corr}")
print(f"Variance of Cov:\n{variance}")

Expected Cov:
[[0.1 0.  0.  0.  0. ]
 [0.  0.1 0.  0.  0. ]
 [0.  0.  0.1 0.  0. ]
 [0.  0.  0.  0.1 0. ]
 [0.  0.  0.  0.  0.1]]
Expected Corr:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
Variance of Cov:
[[0.0025     0.00113636 0.00113636 0.00113636 0.00113636]
 [0.00113636 0.0025     0.00113636 0.00113636 0.00113636]
 [0.00113636 0.00113636 0.0025     0.00113636 0.00113636]
 [0.00113636 0.00113636 0.00113636 0.0025     0.00113636]
 [0.00113636 0.00113636 0.00113636 0.00113636 0.0025    ]]


In [4]:
data, labels = random_data_gen(n_samples=1000, n_feats=10, maha=1., psi_diag=1.)

In [5]:
np.cov(data.T)

array([[  0.2007182 ,  -0.95580753,  -0.07464831,   0.79587807,
         -0.11274548,   1.70641732,   0.9288462 ,  -1.32456017,
         -0.83025096,  -0.77377264],
       [ -0.95580753,   4.7273892 ,   0.36158669,  -3.92550811,
          0.55842923,  -8.42368657,  -4.5875671 ,   6.53652991,
          4.09937494,   3.8183708 ],
       [ -0.07464831,   0.36158669,   0.0330118 ,  -0.30172812,
          0.04204758,  -0.64561534,  -0.35099156,   0.50061088,
          0.31418397,   0.29304502],
       [  0.79587807,  -3.92550811,  -0.30172812,   3.27417942,
         -0.46415232,   7.0063356 ,   3.81405604,  -5.43539216,
         -3.40979621,  -3.17632156],
       [ -0.11274548,   0.55842923,   0.04204758,  -0.46415232,
          0.07216928,  -0.99535321,  -0.54296266,   0.7729501 ,
          0.48457784,   0.45072985],
       [  1.70641732,  -8.42368657,  -0.64561534,   7.0063356 ,
         -0.99535321,  15.04228213,   8.18603897, -11.66605471,
         -7.31753209,  -6.8162875 ],
       [  

In [7]:
data_covar= np.stack([np.cov(random_data_gen(n_samples=200, n_feats=10, maha=1., psi_diag=1., ddof=11)[0][:100].T) for i in range(1000)])

In [8]:
data_corr = np.stack([(D:=np.diag(1/np.sqrt(np.diag(wishart_cov)))) @ wishart_cov @ D for wishart_cov in data_covar])

In [9]:
np.quantile(data_corr, q=[.025, .975], axis=0)

array([[[ 1.        , -0.55795858, -0.60883996, -0.58746159,
         -0.51218008, -0.57995721, -0.58177275, -0.57232002,
         -0.56681246, -0.5560712 ],
        [-0.55795858,  1.        , -0.57296707, -0.56110184,
         -0.53871819, -0.56130545, -0.55770361, -0.53110691,
         -0.55244428, -0.58464229],
        [-0.60883996, -0.57296707,  1.        , -0.53646672,
         -0.51398632, -0.57659415, -0.55571139, -0.55046181,
         -0.59007017, -0.56264305],
        [-0.58746159, -0.56110184, -0.53646672,  1.        ,
         -0.54270057, -0.5316269 , -0.54708017, -0.54963091,
         -0.60527435, -0.55330917],
        [-0.51218008, -0.53871819, -0.51398632, -0.54270057,
          1.        , -0.54404358, -0.5516333 , -0.57946057,
         -0.53218075, -0.56702601],
        [-0.57995721, -0.56130545, -0.57659415, -0.5316269 ,
         -0.54404358,  1.        , -0.53555848, -0.54211642,
         -0.54651206, -0.57432421],
        [-0.58177275, -0.55770361, -0.55571139, -0.5

In [22]:
np.quantile(data_corr, q=[.1, .9], axis=0)

array([[[ 1.        , -0.38559518, -0.38700025, -0.37589341,
         -0.38991168, -0.3675221 , -0.37777729, -0.37870946,
         -0.36564645, -0.38367836],
        [-0.38559518,  1.        , -0.39943433, -0.38765437,
         -0.37406198, -0.35338062, -0.36536066, -0.3935133 ,
         -0.37319157, -0.3811974 ],
        [-0.38700025, -0.39943433,  1.        , -0.41049365,
         -0.39463701, -0.38492749, -0.39160884, -0.38795728,
         -0.38556426, -0.37863835],
        [-0.37589341, -0.38765437, -0.41049365,  1.        ,
         -0.38770109, -0.37313139, -0.37616257, -0.41345547,
         -0.38100565, -0.395552  ],
        [-0.38991168, -0.37406198, -0.39463701, -0.38770109,
          1.        , -0.38165204, -0.39418552, -0.37618871,
         -0.37721229, -0.40002847],
        [-0.3675221 , -0.35338062, -0.38492749, -0.37313139,
         -0.38165204,  1.        , -0.37754554, -0.39649449,
         -0.34929716, -0.38527781],
        [-0.37777729, -0.36536066, -0.39160884, -0.3

### BROKEN: Kronecker product version

In [169]:
import numpy as np

def comm_mat(m, n):
    # determine permutation applied by K
    w = np.arange(m * n).reshape((m, n), order="F").T.ravel(order="F")
    # apply this permutation to the rows (i.e. to each column) of identity matrix and return result
    return np.eye(m * n)[w, :]

def vec(X):
    return np.ravel(X, order='F')

def kron_Var_invwish(psi, dof):
    p = len(psi)
    c2 = ((dof - p)*(dof - p - 1)*(dof - p - 3))**(-1)
    c1 = (dof-p-2)*c2
    c3 = (dof-p-1)**(-2)
    K_pp = comm_mat(p, p)
    return c1 * np.kron(psi, psi) + c2*vec(psi) @ vec(psi).T + c2 * K_pp @ np.kron(psi, psi) 
    